In [0]:
# Azure Event Hub Configuration
event_hub_namespace = "namespace-healthcare"
event_hub_name="healthcare-event-hub"  
event_hub_conn_str = dbutils.secrets.get(scope="healthcarekeyvaultscope", key="event-hub-connection")

kafka_options = {
    'kafka.bootstrap.servers': f"{event_hub_namespace}.servicebus.windows.net:9093",
    'subscribe': event_hub_name,
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.mechanism': 'PLAIN',
    'kafka.sasl.jaas.config': f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{event_hub_conn_str}";',
    'startingOffsets': 'latest',
    'failOnDataLoss': 'false'
}

#Read from eventhub
raw_df = (spark.readStream
          .format("kafka")
          .options(**kafka_options)
          .load()
          )

#Cast data to json
json_df = raw_df.selectExpr("CAST(value AS STRING) as raw_json")


#ADLS configuration 
spark.conf.set(
  "fs.azure.account.key.storageaccounthealthcare.dfs.core.windows.net",
  dbutils.secrets.get(scope="healthcarekeyvaultscope", key="storage-connection")
)

checkpoint_path = "abfss://bronze@storageaccounthealthcare.dfs.core.windows.net/_checkpoints/patient_flow"
bronze_path = "abfss://bronze@storageaccounthealthcare.dfs.core.windows.net/patient_flow"


(
    json_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)  # <-- cloud path
    .start(bronze_path)
)


In [0]:
display(spark.read.format('delta').load(bronze_path))

raw_json
"{""patient_id"": ""0fb2fa94-0015-4f51-8afe-7841af3552a5"", ""gender"": ""Female"", ""age"": 48, ""department"": ""Surgery"", ""admission_time"": ""2026-01-01T17:24:58.229227"", ""discharge_time"": ""2026-01-01T18:24:58.229227"", ""bed_id"": 446, ""hospital_id"": 4}"
"{""patient_id"": ""2488dec3-a16f-487f-8d5c-24f08e51fb84"", ""gender"": ""Female"", ""age"": 64, ""department"": ""Cardiology"", ""admission_time"": ""2026-01-01T02:24:59.234817"", ""discharge_time"": ""2026-01-02T06:24:59.234817"", ""bed_id"": 206, ""hospital_id"": 5}"
"{""patient_id"": ""5a5fdc0c-98d4-48db-9a0d-44227db67d34"", ""gender"": ""Male"", ""age"": 18, ""department"": ""Oncology"", ""admission_time"": ""2025-12-31T08:25:00.236683"", ""discharge_time"": ""2026-01-02T07:25:00.236683"", ""bed_id"": 455, ""hospital_id"": 1}"
"{""patient_id"": ""437345c0-3edf-424d-9b51-4dce1debb9f4"", ""gender"": ""Female"", ""age"": 47, ""department"": ""ICU"", ""admission_time"": ""2025-12-31T15:25:01.242600"", ""discharge_time"": ""2026-01-02T02:25:01.242600"", ""bed_id"": 137, ""hospital_id"": 6}"
"{""patient_id"": ""b0cf19ba-b2a1-40a9-af31-d583de9f1eba"", ""gender"": ""Female"", ""age"": 18, ""department"": ""ICU"", ""admission_time"": ""2026-01-01T01:25:02.249402"", ""discharge_time"": ""2026-01-03T12:25:02.249402"", ""bed_id"": 270, ""hospital_id"": 1}"
"{""patient_id"": ""8b7daa6c-b11e-4545-8099-ada36ce91671"", ""gender"": ""Male"", ""age"": 52, ""department"": ""Pediatrics"", ""admission_time"": ""2025-12-30T10:25:40.419884"", ""discharge_time"": ""2025-12-31T18:25:40.419884"", ""bed_id"": 392, ""hospital_id"": 6}"
"{""patient_id"": ""d4953453-ba8e-40bf-ba46-b602d0a05528"", ""gender"": ""Male"", ""age"": 98, ""department"": ""Surgery"", ""admission_time"": ""2025-12-30T06:25:41.421032"", ""discharge_time"": ""2025-12-30T10:25:41.421032"", ""bed_id"": 451, ""hospital_id"": 6}"
"{""patient_id"": ""77bb1a20-c918-4598-bbf2-59fed3da51ab"", ""gender"": ""Male"", ""age"": 28, ""department"": ""Cardiology"", ""admission_time"": ""2026-01-01T15:25:42.427989"", ""discharge_time"": ""2026-01-02T23:25:42.427989"", ""bed_id"": 433, ""hospital_id"": 4}"
"{""patient_id"": ""a6e4d084-5ca7-4c03-88f9-4bca97177fca"", ""gender"": ""Female"", ""age"": 42, ""department"": ""Maternity"", ""admission_time"": ""2025-12-30T22:24:23.064377"", ""discharge_time"": ""2026-01-01T21:24:23.064377"", ""bed_id"": 483, ""hospital_id"": 5}"
"{""patient_id"": ""fb2af98e-92f7-4f60-b792-e019e17c5c5e"", ""gender"": ""Male"", ""age"": 19, ""department"": ""Emergency"", ""admission_time"": ""2026-01-02T15:24:24.070013"", ""discharge_time"": ""2026-01-04T06:24:24.069824"", ""bed_id"": 250, ""hospital_id"": 7}"
